In [ ]:
# CELL 1: basic setup – install 

# On Kaggle 
!pip install -q pennylane pennylane-lightning torch torchvision

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pennylane as qml
import math
import random

# Reproducibility helpers
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

print("PyTorch version:", torch.__version__)
print("PennyLane version:", qml.__version__)


In [ ]:
!pip install -q "pennylane" "pennylane-lightning[gpu]"

In [ ]:
# CELL 2 configure Torch + PennyLane devices with GPU-first, CPU fallback

# PyTorch device selection
torch_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("PyTorch running on:", torch_device)

# Number of qubits for our QNN
N_QUBITS = 4

# Quantum device selection
try:
    q_device = qml.device("lightning.gpu", wires=N_QUBITS)
    print("PennyLane quantum device: lightning.gpu (GPU)")
except Exception as e:
    print("GPU quantum simulator not available → using default.qubit (CPU).")
    print("Reason:", repr(e))
    q_device = qml.device("default.qubit", wires=N_QUBITS)
    print("PennyLane quantum device: default.qubit (CPU)")


In [ ]:
# CELL 3:  data re-uploading QNN circuit and QNode 

# Hyperparameters 
N_LAYERS = 4 

def qnn_layer(features, weights):
    """One data re-uploading layer: encode features, entangle, then apply trainable rotations."""
    # Encode features (same features on each layer → data re-uploading)
    for i in range(N_QUBITS):
        qml.RY(features[i], wires=i)
        qml.RZ(features[i], wires=i)
    # Entangling ring
    for i in range(N_QUBITS - 1):
        qml.CNOT(wires=[i, i + 1])
    qml.CNOT(wires=[N_QUBITS - 1, 0])
    # Trainable single-qubit rotations
    for i in range(N_QUBITS):
        qml.RY(weights[i, 0], wires=i)
        qml.RZ(weights[i, 1], wires=i)
        qml.RX(weights[i, 2], wires=i)

@qml.qnode(q_device, interface="torch", diff_method="adjoint")
def qnode_qnn(features, weights):
    """Full QNN: stack several data re-uploading layers and measure PauliZ on wire 0."""
    # Expect features to be length N_QUBITS here (we’ll map from raw features later)
    for layer in range(N_LAYERS):
        qnn_layer(features, weights[layer])
    return qml.expval(qml.PauliZ(0))

In [ ]:
# CELL 4: wrap the QNode into a PyTorch nn.Module with a classical feature map

class QuantumPassSuccessQNN(nn.Module):
    def __init__(self, n_features, n_qubits=N_QUBITS, n_layers=N_LAYERS):
        super().__init__()
        assert n_qubits == N_QUBITS, "For now, n_qubits must match N_QUBITS used by the QNode."
        self.n_features = n_features
        self.n_qubits = n_qubits
        self.n_layers = n_layers

        # Linear feature map: arbitrary feature dimension → N_QUBITS (then scaled to angles)
        self.feature_map = nn.Linear(n_features, n_qubits)

        # Trainable quantum circuit parameters: (layers, qubits, 3 rotations per qubit)
        init_scale = 0.01
        self.q_params = nn.Parameter(
            init_scale * torch.randn(n_layers, n_qubits, 3, dtype=torch.float32)
        )

    def forward(self, x):
        # x: [batch_size, n_features]
        x = x.to(torch_device)
        x = self.feature_map(x)                     # [batch_size, n_qubits]
        x = torch.tanh(x) * math.pi                 # keep angles in [-π, π]

        # Evaluate QNode sample-by-sample (simple but clear; OK for moderate dataset sizes)
        outputs = []
        for sample in x:
            out = qnode_qnn(sample, self.q_params)  # scalar expectation value in [-1, 1]
            outputs.append(out)

        outputs = torch.stack(outputs)              # [batch_size]
        # Map [-1, 1] → logits for binary classification (we’ll apply BCEWithLogitsLoss later)
        logits = outputs.unsqueeze(1)               # [batch_size, 1]
        return logits


In [ ]:
# CELL 5: auto-detect StatsBomb dataset folder and list all events directories

import os, json, glob, random
import pandas as pd

BASE_INPUT_DIR = "/kaggle/input"

print("Datasets under /kaggle/input:")
datasets = os.listdir(BASE_INPUT_DIR)
for d in datasets:
    print(" -", d)

# Try to auto-pick a StatsBomb-like folder
sb_candidates = [d for d in datasets if "statsbomb" in d.lower()]
if not sb_candidates:
    raise FileNotFoundError(
        "No dataset with 'statsbomb' in its name found under /kaggle/input.\n"
        "→ In the right sidebar, click '+ Add data' and add the 'StatsBomb Football Data' dataset."
    )

SB_DIR_NAME = sb_candidates[0]
SB_DIR = os.path.join(BASE_INPUT_DIR, SB_DIR_NAME)
print(f"\nUsing StatsBomb dataset folder: {SB_DIR}")

# Find all directories named 'events' inside this dataset
event_dirs = []
for root, dirs, files in os.walk(SB_DIR):
    if os.path.basename(root).lower() == "events":
        event_dirs.append(root)

if not event_dirs:
    raise FileNotFoundError(
        f"No 'events' directories found inside {SB_DIR}.\n"
        "Check the dataset structure or confirm that it really contains StatsBomb event JSONs."
    )

print("\nFound events directories:")
for ed in event_dirs:
    print(" -", ed)


In [ ]:
# CELL 6 (FULLY FIXED): extract pass events safely + reservoir sampling

import numpy as np

def safe_get(d, key, default=None):
    """Returns d[key] if d is a dict, otherwise returns default."""
    if isinstance(d, dict):
        return d.get(key, default)
    return default

def safe_list(v, length=2):
    """Ensures v is a list of given length (pads with None)."""
    if isinstance(v, list):
        out = v + [None]*length
        return out[:length]
    return [None]*length

MAX_PASSES = 200_000

pass_rows = []
total_pass_seen = 0

for ed in event_dirs:
    event_files = glob.glob(os.path.join(ed, "*.json"))
    print(f"Processing {len(event_files)} event files from: {ed}")

    for ef in event_files:
        with open(ef, "r") as f:
            events = json.load(f)

        for e in events:

            # Filter only Pass events
            if safe_get(e, "type", {}).get("name") != "Pass":
                continue

            total_pass_seen += 1

            # SAFE extraction of all fields
            team_name   = safe_get(safe_get(e, "team"),   "name")
            player_name = safe_get(safe_get(e, "player"), "name")

            minute = safe_get(e, "minute")
            second = safe_get(e, "second")
            period = safe_get(safe_get(e, "period"), "id")

            loc = safe_list(safe_get(e, "location", [None, None]), 2)
            p   = safe_get(e, "pass", {}) or {}

            end_loc = safe_list(p.get("end_location", [None, None]), 2)

            height_info  = safe_get(p, "height", {})
            outcome_info = safe_get(p, "outcome", {})

            row = {
                "team": team_name,
                "player": player_name,
                "minute": minute,
                "second": second,
                "period": period,
                "start_x": loc[0],
                "start_y": loc[1],
                "end_x": end_loc[0],
                "end_y": end_loc[1],
                "length": safe_get(p, "length"),
                "angle": safe_get(p, "angle"),
                "height": safe_get(height_info, "name"),
                "under_pressure": bool(e.get("under_pressure", False)),
                "outcome": safe_get(outcome_info, "name"),  # None or 'Complete' = success
            }

            # Reservoir sampling
            if len(pass_rows) < MAX_PASSES:
                pass_rows.append(row)
            else:
                j = np.random.randint(0, total_pass_seen)
                if j < MAX_PASSES:
                    pass_rows[j] = row

print("\nTotal pass events seen:", total_pass_seen)
print("Total pass events kept:", len(pass_rows))

df_passes = pd.DataFrame(pass_rows)
print("\ndf_passes shape:", df_passes.shape)
print("Outcome counts:", df_passes['outcome'].value_counts(dropna=False).head(10))
df_passes.head()


In [ ]:
# CELL 7: build numeric feature matrix + success labels from df_passes

from math import sqrt

# 1) Define success label: None or 'Complete' = success (1), everything else = failure (0)
outcome_str = df_passes["outcome"].astype(str).str.lower()
success_mask = df_passes["outcome"].isna() | (outcome_str == "complete")
df_passes["label_success"] = success_mask.astype(int)

print("Pass success rate:", df_passes["label_success"].mean())

# 2) Encode pass height as numeric (Ground/Low/High → 0/1/2; others → -1)
height_map = {"ground pass": 0, "low pass": 1, "high pass": 2}
height_code = df_passes["height"].astype(str).str.lower().map(height_map).fillna(-1)

# 3) Basic geometry
start_x = df_passes["start_x"].astype(float)
start_y = df_passes["start_y"].astype(float)
end_x   = df_passes["end_x"].astype(float)
end_y   = df_passes["end_y"].astype(float)
length  = df_passes["length"].astype(float)
angle   = df_passes["angle"].astype(float)

# 4) Derived geometry features
dx = end_x - start_x
dy = end_y - start_y

# Assume StatsBomb pitch: 120 x 80, goal at (120, 40)
GOAL_X, GOAL_Y = 120.0, 40.0
dist_start_to_goal = ((GOAL_X - start_x)**2 + (GOAL_Y - start_y)**2) ** 0.5
dist_end_to_goal   = ((GOAL_X - end_x)**2 + (GOAL_Y - end_y)**2) ** 0.5
forward_progress   = end_x - start_x  # positive if pass moves ball towards opponent goal

# 5) Context features
minute = df_passes["minute"].astype(float)
second = df_passes["second"].astype(float)
period = df_passes["period"].astype(float)
under_pressure = df_passes["under_pressure"].astype(bool).astype(float)

# 6) Stack all numeric features into a matrix
feature_cols = [
    start_x, start_y,
    end_x, end_y,
    length, angle,
    dx, dy,
    dist_start_to_goal, dist_end_to_goal,
    forward_progress,
    height_code.astype(float),
    under_pressure,
    minute, second, period,
]

X_np = np.vstack([col.values for col in feature_cols]).T  # shape [n_samples, n_features]
y_np = df_passes["label_success"].astype(np.float32).values

print("Feature matrix shape:", X_np.shape)
print("Label vector shape:", y_np.shape)


In [ ]:
# CELL 8: scale features, split data, build tensors, and initialize QNN model

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 1) Train/val/test split (on raw features to avoid leakage)
X_train, X_temp, y_train, y_temp = train_test_split(
    X_np, y_np, test_size=0.3, random_state=SEED, stratify=y_np
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=SEED, stratify=y_temp
)

print("Raw splits shapes:",
      "train", X_train.shape, "val", X_val.shape, "test", X_test.shape)

# 2) Fit scaler on TRAIN ONLY, then transform all splits
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)

print("Train feature means (approx):", X_train_scaled.mean(axis=0)[:5])
print("Train feature stds (approx):",  X_train_scaled.std(axis=0)[:5])

# 3) Convert to torch tensors
X_train_t = torch.tensor(X_train_scaled, dtype=torch.float32).to(torch_device)
y_train_t = torch.tensor(y_train,        dtype=torch.float32).unsqueeze(1).to(torch_device)

X_val_t = torch.tensor(X_val_scaled, dtype=torch.float32).to(torch_device)
y_val_t = torch.tensor(y_val,        dtype=torch.float32).unsqueeze(1).to(torch_device)

X_test_t = torch.tensor(X_test_scaled, dtype=torch.float32).to(torch_device)
y_test_t = torch.tensor(y_test,        dtype=torch.float32).unsqueeze(1).to(torch_device)

# 4) Build QNN model and initialize weights
n_features = X_train_t.shape[1]
q_model = QuantumPassSuccessQNN(n_features=n_features).to(torch_device)

# Initialize classical and quantum weights
for m in q_model.modules():
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)
with torch.no_grad():
    q_model.q_params.copy_(0.01 * torch.randn_like(q_model.q_params))

print("QNN model ready with", n_features, "input features on", torch_device)
q_model


In [ ]:
# CELL 9: create a balanced 50/50 subset with at most 10k samples

MAX_TOTAL_SAMPLES = 20_000  # cap total samples used for QNN training

# y_np and X_np come from CELL 7 (full 200k feature matrix + labels)
assert X_np.shape[0] == y_np.shape[0]

idx_success = np.where(y_np == 1)[0]
idx_failure = np.where(y_np == 0)[0]

n_success = len(idx_success)
n_failure = len(idx_failure)
print("Original counts → success:", n_success, "failure:", n_failure)

# how many per class we can keep
max_per_class = MAX_TOTAL_SAMPLES // 2
n_per_class = min(n_success, n_failure, max_per_class)
print("Using per-class samples:", n_per_class, "(total =", 2 * n_per_class, ")")

rng = np.random.default_rng(SEED)
idx_success_sub = rng.choice(idx_success, size=n_per_class, replace=False)
idx_failure_sub = rng.choice(idx_failure, size=n_per_class, replace=False)

idx_balanced = np.concatenate([idx_success_sub, idx_failure_sub])
rng.shuffle(idx_balanced)

X_small = X_np[idx_balanced]
y_small = y_np[idx_balanced].astype(np.float32)

print("Balanced subset shapes:", X_small.shape, y_small.shape)
print("Balanced success rate (should be ~0.5):", y_small.mean())


In [ ]:
# CELL 10: scale balanced features, split, build tensors, and initialize QNN

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 1) train/val/test split on balanced subset
X_train, X_temp, y_train, y_temp = train_test_split(
    X_small, y_small, test_size=0.3, random_state=SEED, stratify=y_small
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=SEED, stratify=y_temp
)

print("Balanced splits:",
      "train", X_train.shape, "val", X_val.shape, "test", X_test.shape)

# 2) fit scaler on TRAIN only, then transform all splits
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)

print("Train means (≈0):", X_train_scaled.mean(axis=0)[:5])
print("Train stds  (≈1):", X_train_scaled.std(axis=0)[:5])

# 3) convert to torch tensors
X_train_t = torch.tensor(X_train_scaled, dtype=torch.float32).to(torch_device)
y_train_t = torch.tensor(y_train,        dtype=torch.float32).unsqueeze(1).to(torch_device)

X_val_t = torch.tensor(X_val_scaled, dtype=torch.float32).to(torch_device)
y_val_t = torch.tensor(y_val,        dtype=torch.float32).unsqueeze(1).to(torch_device)

X_test_t = torch.tensor(X_test_scaled, dtype=torch.float32).to(torch_device)
y_test_t = torch.tensor(y_test,        dtype=torch.float32).unsqueeze(1).to(torch_device)

# 4) build QNN model and initialize weights fresh
n_features = X_train_t.shape[1]
q_model = QuantumPassSuccessQNN(n_features=n_features).to(torch_device)

for m in q_model.modules():
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)

with torch.no_grad():
    q_model.q_params.copy_(0.01 * torch.randn_like(q_model.q_params))

print("QNN model ready with", n_features, "features on", torch_device)
q_model


In [ ]:
# CELL 11: sanitize scaled arrays and create PyTorch DataLoaders

from torch.utils.data import TensorDataset, DataLoader

# Clean NaNs and infs in scaled arrays
X_train_scaled = np.nan_to_num(X_train_scaled, nan=0.0, posinf=0.0, neginf=0.0)
X_val_scaled   = np.nan_to_num(X_val_scaled,   nan=0.0, posinf=0.0, neginf=0.0)
X_test_scaled  = np.nan_to_num(X_test_scaled,  nan=0.0, posinf=0.0, neginf=0.0)

# Rebuild tensors (overwrite)
X_train_t = torch.tensor(X_train_scaled, dtype=torch.float32).to(torch_device)
y_train_t = torch.tensor(y_train,        dtype=torch.float32).unsqueeze(1).to(torch_device)

X_val_t = torch.tensor(X_val_scaled, dtype=torch.float32).to(torch_device)
y_val_t = torch.tensor(y_val,        dtype=torch.float32).unsqueeze(1).to(torch_device)

X_test_t = torch.tensor(X_test_scaled, dtype=torch.float32).to(torch_device)
y_test_t = torch.tensor(y_test,        dtype=torch.float32).unsqueeze(1).to(torch_device)

# DataLoaders
BATCH_SIZE = 32

train_loader = DataLoader(TensorDataset(X_train_t, y_train_t), batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(TensorDataset(X_val_t,   y_val_t),   batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(TensorDataset(X_test_t,  y_test_t),  batch_size=BATCH_SIZE, shuffle=False)

len(train_loader), len(val_loader), len(test_loader)


In [ ]:
# CELL 12 (UPDATED): training loop with tqdm progress bar, best checkpoint, and loss history

from tqdm.notebook import tqdm

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(q_model.parameters(), lr=0.01)

EPOCHS = 15
BEST_MODEL_PATH = "best_qnn_pass_success.pt"

train_iter_losses = []   # per-iteration training losses (for plotting)
val_epoch_losses = []    # per-epoch validation losses
val_epoch_accs = []      # per-epoch validation accuracies

best_val_acc = 0.0

def eval_on_loader(model, loader):
    model.eval()
    total_loss = 0.0
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for xb, yb in loader:
            logits = model(xb)
            loss = criterion(logits, yb)
            total_loss += loss.item() * xb.size(0)
            preds = (torch.sigmoid(logits) >= 0.5).float()
            total_correct += (preds == yb).sum().item()
            total_samples += xb.size(0)
    return total_loss / total_samples, total_correct / total_samples

for epoch in range(1, EPOCHS + 1):
    q_model.train()
    running_loss = 0.0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}", leave=False)
    for xb, yb in pbar:
        optimizer.zero_grad()
        logits = q_model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        batch_loss = loss.item()
        running_loss += batch_loss * xb.size(0)
        train_iter_losses.append(batch_loss)

        pbar.set_postfix(loss=f"{batch_loss:.4f}")

    train_loss = running_loss / len(train_loader.dataset)
    val_loss, val_acc = eval_on_loader(q_model, val_loader)

    val_epoch_losses.append(val_loss)
    val_epoch_accs.append(val_acc)

    # Save best model by validation accuracy
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(q_model.state_dict(), BEST_MODEL_PATH)

    print(f"Epoch {epoch:02d} | train_loss={train_loss:.4f} | "
          f"val_loss={val_loss:.4f} | val_acc={val_acc:.4f} | best_val_acc={best_val_acc:.4f}")

# Load best weights before final test evaluation
print(f"\nLoading best model from: {BEST_MODEL_PATH}")
q_model.load_state_dict(torch.load(BEST_MODEL_PATH, map_location=torch_device))

test_loss, test_acc = eval_on_loader(q_model, test_loader)
print(f"Final TEST → loss={test_loss:.4f} | acc={test_acc:.4f}")


In [ ]:
# CELL 13: plot training (per-iteration) loss and validation (per-epoch) loss

import matplotlib.pyplot as plt
import numpy as np

# Training loss vs iterations
plt.figure(figsize=(8, 4))
plt.plot(range(len(train_iter_losses)), train_iter_losses)
plt.xlabel("Iteration")
plt.ylabel("Train loss (BCE)")
plt.title("Training loss vs iterations")
plt.grid(True)
plt.show()

# Validation loss vs epochs
plt.figure(figsize=(6, 4))
plt.plot(range(1, len(val_epoch_losses) + 1), val_epoch_losses, marker="o")
plt.xlabel("Epoch")
plt.ylabel("Validation loss (BCE)")
plt.title("Validation loss vs epochs")
plt.grid(True)
plt.show()


In [ ]:
# CELL 14 (FIXED): correct path to the uploaded model file

MODEL_DIR = "/kaggle/input/best-qnn-pass-success-pt"
MODEL_PATH = os.path.join(MODEL_DIR, "best_qnn_pass_success.pt")

print("Model path:", MODEL_PATH)

if not os.path.isfile(MODEL_PATH):
    raise FileNotFoundError(f"Model file not found at {MODEL_PATH}")


In [ ]:
# CELL 15: load pretrained QNN weights and evaluate on the test set

q_model.load_state_dict(torch.load(MODEL_PATH, map_location=torch_device))
q_model.to(torch_device)
q_model.eval()

print("Loaded model from:", MODEL_PATH)

# If eval_on_loader isn't available:
criterion = nn.BCEWithLogitsLoss()

def eval_on_loader(model, loader):
    model.eval()
    total_loss, total_correct, total_samples = 0.0, 0, 0
    with torch.no_grad():
        for xb, yb in loader:
            logits = model(xb)
            loss = criterion(logits, yb)
            preds = (torch.sigmoid(logits) >= 0.5).float()
            total_loss += loss.item() * xb.size(0)
            total_correct += (preds == yb).sum().item()
            total_samples += xb.size(0)
    return total_loss / total_samples, total_correct / total_samples

test_loss, test_acc = eval_on_loader(q_model, test_loader)
print(f"Restored model → loss={test_loss:.4f} | acc={test_acc:.4f}")


In [ ]:
# CELL 16: feature names and helper for QNN probabilities

import numpy as np
import matplotlib.pyplot as plt

# Must match the order we used when building X_np in CELL 7
feature_names = [
    "start_x", "start_y",
    "end_x", "end_y",
    "length", "angle",
    "dx", "dy",
    "dist_start_to_goal", "dist_end_to_goal",
    "forward_progress",
    "height_code",
    "under_pressure",
    "minute", "second", "period",
]

assert len(feature_names) == X_train.shape[1]

def qnn_probs_from_raw(X_raw):
    """
    X_raw: numpy array [n_samples, n_features] in ORIGINAL feature space (not scaled).
    Returns: numpy array of probabilities P(success) via sigmoid(q_model(x)).
    """
    X_scaled = scaler.transform(X_raw)                     # scale with train-fitted scaler
    X_t = torch.tensor(X_scaled, dtype=torch.float32).to(torch_device)
    with torch.no_grad():
        logits = q_model(X_t)
        probs = torch.sigmoid(logits).cpu().numpy().reshape(-1)
    return probs


In [ ]:
# CELL 17: decision boundary over (length, forward_progress)

# pick the two features we want to visualize
f1_name = "length"
f2_name = "forward_progress"

f1_idx = feature_names.index(f1_name)
f2_idx = feature_names.index(f2_name)

# use original (unscaled) validation subset for ranges
X_val_orig = X_val  # from CELL 10, before scaling

f1_vals = X_val_orig[:, f1_idx]
f2_vals = X_val_orig[:, f2_idx]

f1_min, f1_max = np.percentile(f1_vals, [2, 98])
f2_min, f2_max = np.percentile(f2_vals, [2, 98])

# grid in original feature space
grid_size = 60
f1_grid = np.linspace(f1_min, f1_max, grid_size)
f2_grid = np.linspace(f2_min, f2_max, grid_size)

# base point = mean of train features (original space)
base_vec = X_train.mean(axis=0)

grid_points = []
for v1 in f1_grid:
    for v2 in f2_grid:
        x = base_vec.copy()
        x[f1_idx] = v1
        x[f2_idx] = v2
        grid_points.append(x)

grid_points = np.stack(grid_points, axis=0)  # [grid_size^2, n_features]

# get QNN probabilities on the grid
probs = qnn_probs_from_raw(grid_points)
probs_grid = probs.reshape(grid_size, grid_size)

# plot contour + real points (validation set)
plt.figure(figsize=(7, 5))
cs = plt.contourf(f1_grid, f2_grid, probs_grid.T, levels=20, alpha=0.7)
plt.colorbar(cs, label="P(success)")

# overlay validation samples (downsample for clarity)
idx_sample = np.random.choice(len(X_val_orig), size=min(1000, len(X_val_orig)), replace=False)
plt.scatter(
    X_val_orig[idx_sample, f1_idx],
    X_val_orig[idx_sample, f2_idx],
    c=y_val[idx_sample],
    edgecolors="k",
    linewidths=0.3,
    alpha=0.7,
)
plt.xlabel(f1_name)
plt.ylabel(f2_name)
plt.title("QNN decision surface: pass length vs forward progress")
plt.grid(True)
plt.show()


In [ ]:
# CELL 18: permutation feature importance based on validation accuracy

from copy import deepcopy

# baseline accuracy on current val_loader
baseline_val_loss, baseline_val_acc = eval_on_loader(q_model, val_loader)
print(f"Baseline val_acc = {baseline_val_acc:.4f}")

importances = []

# work with numpy copy of scaled validation features
X_val_scaled_np = X_val_scaled.copy()

for j, fname in enumerate(feature_names):
    X_perm = X_val_scaled_np.copy()
    # permute column j
    perm_indices = np.random.permutation(X_perm.shape[0])
    X_perm[:, j] = X_perm[perm_indices, j]

    # build a temporary DataLoader
    X_perm_t = torch.tensor(X_perm, dtype=torch.float32).to(torch_device)
    y_val_t_local = torch.tensor(y_val, dtype=torch.float32).unsqueeze(1).to(torch_device)
    perm_loader = DataLoader(TensorDataset(X_perm_t, y_val_t_local), batch_size=32, shuffle=False)

    _, perm_acc = eval_on_loader(q_model, perm_loader)
    importance = baseline_val_acc - perm_acc
    importances.append(importance)
    print(f"{fname:20s} | acc={perm_acc:.4f} | drop={importance:.4f}")

# bar plot of importance
plt.figure(figsize=(10, 4))
order = np.argsort(importances)[::-1]
plt.bar(range(len(feature_names)), np.array(importances)[order])
plt.xticks(range(len(feature_names)), np.array(feature_names)[order], rotation=60, ha="right")
plt.ylabel("Accuracy drop when permuted")
plt.title("Permutation feature importance (validation set)")
plt.tight_layout()
plt.grid(True, axis="y")
plt.show()


In [ ]:
# CELL 21: install gradio and define QNN prediction helper for the mini app

!pip install -q gradio

import gradio as gr

GOAL_X, GOAL_Y = 120.0, 40.0  # same as before

# map height string to numeric code, consistent with training
height_map = {"Ground Pass": 0, "Low Pass": 1, "High Pass": 2}

def predict_pass_success(start_x, start_y,
                         end_x, end_y,
                         height_str,
                         under_pressure,
                         minute, second, period):
    """
    Build the 16D feature vector from UI inputs, scale, and run QNN to get P(success).
    """

    # 1) geometry
    dx = end_x - start_x
    dy = end_y - start_y
    length = (dx**2 + dy**2) ** 0.5
    # avoid divide-by-zero; if length=0, set angle=0
    if length > 0:
        angle = np.arctan2(dy, dx)
    else:
        angle = 0.0

    dist_start_to_goal = ((GOAL_X - start_x)**2 + (GOAL_Y - start_y)**2) ** 0.5
    dist_end_to_goal   = ((GOAL_X - end_x)**2   + (GOAL_Y - end_y)**2) ** 0.5
    forward_progress   = end_x - start_x

    # 2) height + context
    height_code = height_map.get(height_str, -1)
    up_flag = 1.0 if under_pressure else 0.0

    # 3) pack into the same feature order used for training (original space)
    x_raw = np.array([
        start_x, start_y,
        end_x,   end_y,
        length,  angle,
        dx, dy,
        dist_start_to_goal, dist_end_to_goal,
        forward_progress,
        float(height_code),
        up_flag,
        float(minute), float(second), float(period),
    ], dtype=np.float32).reshape(1, -1)

    # 4) scale and run QNN (using existing scaler & q_model)
    x_scaled = scaler.transform(x_raw)
    x_scaled = np.nan_to_num(x_scaled, nan=0.0, posinf=0.0, neginf=0.0)

    x_t = torch.tensor(x_scaled, dtype=torch.float32).to(torch_device)
    q_model.eval()
    with torch.no_grad():
        logits = q_model(x_t)
        prob = torch.sigmoid(logits).cpu().numpy().reshape(-1)[0]

    # 5) human-readable message
    label = "Likely SUCCESS ✅" if prob >= 0.5 else "Likely FAILURE ❌"
    return {
        "Probability of success": float(prob),
        "Prediction": label,
    }


In [ ]:
# CELL 22 (FIXED): Gradio UI for interactive QNN pass-success testing

with gr.Blocks() as demo:
    gr.Markdown("## Quantum Pass Success Predictor ⚽🧠")
    gr.Markdown(
        "Adjust the pass parameters below (StatsBomb 120×80 pitch) and let the QNN "
        "estimate the probability that the pass will be completed."
    )

    with gr.Row():
        with gr.Column():
            start_x = gr.Slider(0, 120, value=60, label="Start X (0–120)")
            start_y = gr.Slider(0, 80,  value=40, label="Start Y (0–80)")
            end_x   = gr.Slider(0, 120, value=80, label="End X (0–120)")
            end_y   = gr.Slider(0, 80,  value=40, label="End Y (0–80)")

        with gr.Column():
            height_str = gr.Radio(
                choices=["Ground Pass", "Low Pass", "High Pass"],
                value="Ground Pass",
                label="Pass height"
            )
            under_pressure = gr.Checkbox(False, label="Under pressure?")
            minute = gr.Slider(0, 120, value=30, step=1, label="Minute")
            second = gr.Slider(0, 59, value=0, step=1, label="Second")
            period = gr.Slider(1, 4, value=1, step=1, label="Period (1–4)")

    output = gr.JSON(label="QNN output")

    btn = gr.Button("Predict pass success")
    btn.click(
        predict_pass_success,
        inputs=[start_x, start_y, end_x, end_y,
                height_str, under_pressure,
                minute, second, period],
        outputs=output,
    )

# IMPORTANT for Kaggle: use share=True and open the printed https://...gradio.live link
demo.launch(share=True, inline=False)


In [ ]:
# CELL 23: mini test program to inspect real test passes and QNN predictions

def describe_pass(x_raw, y_true, prob):
    """Pretty-print a single pass in football terms + QNN prediction."""
    d = dict(zip(feature_names, x_raw))

    desc = []
    desc.append(f"  Start: ({d['start_x']:.1f}, {d['start_y']:.1f})")
    desc.append(f"  End:   ({d['end_x']:.1f}, {d['end_y']:.1f})")
    desc.append(f"  Length: {d['length']:.1f} m, forward_progress: {d['forward_progress']:.1f} m")
    desc.append(f"  Dist to goal (start→end): {d['dist_start_to_goal']:.1f} → {d['dist_end_to_goal']:.1f} m")
    hcode = int(d["height_code"])
    hname = {0: "Ground", 1: "Low", 2: "High"}.get(hcode, f"code={hcode}")
    desc.append(f"  Height: {hname}, under_pressure: {bool(round(d['under_pressure']))}")
    desc.append(f"  Time: minute={d['minute']:.0f}, second={d['second']:.0f}, period={d['period']:.0f}")
    desc.append(f"  TRUE label: {'SUCCESS' if y_true==1 else 'FAILURE'}")
    desc.append(f"  QNN P(success): {prob:.3f}  →  {'SUCCESS' if prob>=0.5 else 'FAILURE'}")
    return "\n".join(desc)

def test_random_passes(n_samples=5):
    """Sample n real test passes and show what the QNN predicts."""
    # X_test, y_test are ORIGINAL (unscaled) test features/labels
    idxs = np.random.choice(len(X_test), size=min(n_samples, len(X_test)), replace=False)
    for i, idx in enumerate(idxs, start=1):
        x_raw = X_test[idx]
        y_true = int(y_test[idx])

        # scale & predict with QNN
        x_scaled = scaler.transform(x_raw.reshape(1, -1))
        x_scaled = np.nan_to_num(x_scaled, nan=0.0, posinf=0.0, neginf=0.0)
        x_t = torch.tensor(x_scaled, dtype=torch.float32).to(torch_device)
        q_model.eval()
        with torch.no_grad():
            prob = torch.sigmoid(q_model(x_t)).cpu().numpy().reshape(-1)[0]

        print(f"=== Sample {i} (test index {idx}) ===")
        print(describe_pass(x_raw, y_true, prob))
        print()

# Run a quick check
test_random_passes(5)
